<a href="https://colab.research.google.com/github/ellozam/ciencia-datos-notebooks/blob/main/proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Instalar librerías necesarias
!pip install gradio openpyxl --quiet

import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
import glob
import numpy as np
from io import BytesIO
import tempfile
import openpyxl
import os

# Buscar archivos CSV del Banco Mundial
csv_path = glob.glob("/content/API_*.csv")
if not csv_path:
    raise FileNotFoundError("❌ No se encontró archivo CSV en /content/. Por favor sube uno con nombre tipo 'API_*.csv'.")

csv_file = csv_path[0]
print(f"✅ Usando archivo: {csv_file}")

# Leer y preparar datos
df = pd.read_csv(csv_file, skiprows=4)
columnas = ['Country Name'] + [str(year) for year in range(2000, 2023)]
df = df[columnas].dropna()
paises = sorted(df['Country Name'].unique().tolist())
matriz_pib = df.set_index('Country Name').apply(pd.to_numeric, errors='coerce')

# Función principal
def comparar_y_exportar(pais1, pais2):
    try:
        datos1 = matriz_pib.loc[pais1].dropna()
        datos2 = matriz_pib.loc[pais2].dropna()

        comparativa = pd.DataFrame({
            'Año': datos1.index.astype(int),
            pais1: datos1.values,
            pais2: datos2.reindex(datos1.index).values
        })

        comparativa = comparativa.replace([np.inf, -np.inf], 0).fillna(0)
        comparativa[f"{pais1} Crec %"] = comparativa[pais1].pct_change()
        comparativa[f"{pais2} Crec %"] = comparativa[pais2].pct_change()
        comparativa = comparativa.round(2)

        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(comparativa['Año'], comparativa[pais1] / 1e9, label=pais1, marker='o')
        ax.plot(comparativa['Año'], comparativa[pais2] / 1e9, label=pais2, marker='o')
        ax.set_title(f'PIB: {pais1} vs {pais2}')
        ax.set_xlabel('Año')
        ax.set_ylabel('PIB (miles de millones USD)')
        ax.grid(True)
        ax.legend()
        plt.tight_layout()

        imagen = BytesIO()
        fig.savefig(imagen, format='png')
        imagen.seek(0)

        tmp_path = tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx").name

        with pd.ExcelWriter(tmp_path, engine='openpyxl') as writer:
            comparativa.to_excel(writer, index=False, sheet_name='Comparación PIB')
            workbook = writer.book
            worksheet = workbook['Comparación PIB']

            for col_num, value in enumerate(comparativa.columns.values):
                worksheet.cell(row=1, column=col_num + 1, value=value)
                worksheet.column_dimensions[openpyxl.utils.get_column_letter(col_num + 1)].width = 18

            for row_num, row in comparativa.iterrows():
                for col_num, value in enumerate(row):
                    cell = worksheet.cell(row=row_num + 2, column=col_num + 1, value=value)
                    if "Crec" in comparativa.columns[col_num]:
                        cell.number_format = '0.00%'

            image_path = "/tmp/grafico.png"
            fig.savefig(image_path)
            worksheet.add_image(openpyxl.drawing.image.Image(image_path), 'H2')

        # IA: sugerencias de países similares
        def sugerir(pais_ref):
            prom = matriz_pib.loc[pais_ref].mean()
            dist = matriz_pib.apply(lambda x: np.abs(x.mean() - prom), axis=1)
            return dist.sort_values().drop(index=pais_ref).head(3).index.tolist()

        sugeridos1 = sugerir(pais1)
        sugeridos2 = sugerir(pais2)

        texto = f"""🤖 IA Sugerencias:
- Países similares a **{pais1}**: {', '.join(sugeridos1)}
- Países similares a **{pais2}**: {', '.join(sugeridos2)}"""

        return fig, comparativa, texto, tmp_path, ""

    except Exception as e:
        return None, None, f"❌ Error: {str(e)}", None, "⚠️ Ocurrió un error al comparar."

# Interfaz Gradio mejorada con retorno corregido
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🌍 Comparador de PIB Mundial\nUna herramienta interactiva con IA y exportación a Excel.")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🔽 Selecciona los países para comparar")
            dropdown1 = gr.Dropdown(choices=paises, label="📌 País 1", value=paises[0])
            dropdown2 = gr.Dropdown(choices=paises, label="📌 País 2", value=paises[1])
            boton = gr.Button("📊 Comparar PIB", variant='primary')
            estado = gr.Markdown("")  # Mensaje de estado dinámico

        with gr.Column(scale=2):
            gr.Markdown("### 📈 Gráfico de comparación del PIB")
            grafico = gr.Plot(label="PIB comparado")
            gr.Markdown("### 📋 Tabla de datos comparativos")
            tabla = gr.Dataframe(label="Datos comparados", interactive=False)

    with gr.Row():
        with gr.Column():
            texto = gr.Markdown("### 🤖 IA Sugerencias de países similares")
        with gr.Column():
            archivo = gr.File(label="📥 Descargar Excel generado", file_types=[".xlsx"])

    # Esta función maneja los retornos de forma limpia
    def wrapper(p1, p2):
        fig, df_out, tips, file_out, estado_msg = comparar_y_exportar(p1, p2)
        return fig, df_out, tips, file_out, estado_msg

    boton.click(fn=wrapper,
                inputs=[dropdown1, dropdown2],
                outputs=[grafico, tabla, texto, archivo, estado])

# Lanzar app
demo.launch(share=True)




✅ Usando archivo: /content/API_NY.GDP.MKTP.CD_DS2_es_csv_v2_1992.csv
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6c745d3693ec640ee3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
